In [3]:
import pypandoc
output = pypandoc.convert_file('Primat_s.docx', to='html', outputfile="Primat_s.html", extra_args=['-s'])

In [13]:
#!/usr/bin/env python
from lxml import etree
import lxml.html
import pprint
import re
import os
import cssselect
import csv
from IPython.display import display, HTML
# вручную надо удалить всё после «Географические синонимы»
# придумать, как делать дрилл-даун до исходных данных
# исключать из рассмотрения Insertae sedis
from parglare import Parser, Grammar

g = Grammar.from_file("grammar.pg", ignore_case=True)
actions = {
    "Tokens":[lambda _, nodes: nodes[0] if nodes[0] else nodes[1]], # предпочитаем первый
    "Token": [lambda _, nodes: nodes[1],
              lambda _, nodes: nodes[0],
              lambda _, value: [],
              lambda _, value: []],
    "Layer": [lambda _, value: [145.0, 66.0],  #Мел
              lambda _, value: [ 66.0,  56.0], #Палеоцен
              lambda _, value: [ 56.0,  33.9], #Эоцен
              lambda _, value: [ 33.9,  23.03],
              lambda _, value: [ 23.03,  5.333],
              lambda _, value: [ 5.333,  1.806],
              lambda _, value: [ 1.806,  0.0117],
              lambda _, value: [ 0.0117, 0.0]],
    "LayerPrefix": [lambda _, value: [],
              lambda _, value: [],
              lambda _, value: []]
}
parser = Parser(g, debug=False, debug_trace=True, actions=actions)


taxons = [
    "надотряд\\b",
    "отряд\\b",
    "подотряд\\b",
    "гипотряд\\b",
    "инфраотряд\\b",
    "парвотряд\\b",
    "надсем\\b\\.",
    "сем\\b\\.",
    "подсем\\b\\.",
#    r"триба\b",
]

taxonBreadcrumbs = ["","","","","","","","",""]

tree = lxml.html.parse("Primat_s.html").getroot()

for e in tree.cssselect("span, ul, ol, li, a, font"): # TODO новый ХТМЛ формат может и не надо так чистить
    e.drop_tag()

# информация для дебага. Добавляю ссылки, по которым можно будет скроллить документ. Эти номера также попадут в таблицу с данными.
for index, e in enumerate(tree.cssselect("p, h1, h2, h3, h4, h5, h6, table tr td")):
    anchorStr = "debug"+str(index)
    anchorElement = etree.Element("a", name="debug"+str(index))
    anchorElement.set("class", "debug")
    anchorElement.tail = e.text
    e.text = None
    e.insert(0, anchorElement)# = anchorStr + e.text if e.text else anchorStr



In [24]:
with open('out.tsv', 'w') as outCSVfile:
    outCSV = csv.writer(outCSVfile, quoting=csv.QUOTE_ALL, dialect=csv.excel_tab)
    outCSV.writerow(taxons + ["род", "period_start", "period_end", "debug"])
    
    for e in tree.cssselect("p, h1, h2, h3, h4, h5, h6, table tr"):
        #line = re.sub(r"\t", " ", line)
        if e.tag == "tr" and len(e)==3: # trying to grab the period
            # считываем название рода. Берём первое слово со знаками
            genusStr = e[0].text_content()
            genusStr = re.sub(r"\n|\t", r" ", genusStr)
            if re.match(r"^(\S+).*", genusStr, flags=re.DOTALL):
                genusStr = re.sub(r"^(\S+\s+\S*).*", r"\1", genusStr) #re.DOTALL чтобы обрезать многострочный текст до конца
                if genusStr:
                    # считываем и парсим период
                    periodStr = e[1].text_content()
                    periodStr = re.sub(r"\n|\t", " ", periodStr)
                    periodStr = re.sub(r"\(.*\)", r"", periodStr)
                    periodStr = re.sub(r"(\d),(\d)", r"\1.\2", periodStr) # changing digit separator: 0,1 to 0.1
                    period = parser.parse(periodStr)
                    if not period:
                        period = ["",""]
                    debugStr = e[0][0].get("name")
                    outCSV.writerow(taxonBreadcrumbs + [str(genusStr)] + period + [str(debugStr)])
                    #print(outLine)
                    #outCSV = outCSV + os.linesep + outLine
                    pass
        else: # if it's p, h1 and so on
            paragraphStr = e.text_content()
            paragraphStr = re.sub(r"\n|\t", r" ", paragraphStr)
            for index, t in enumerate(taxons):
                #match = re.sub(r"^\s*("+t+".*)", "* "*index+r"* \1", line, flags=re.IGNORECASE)
                #print(paragraphStr)
                if re.search(r"^\s*("+t+".*)", paragraphStr, flags=re.IGNORECASE):
                    # Когда спускаемся по цепочке таксонов, запоминаем все, что выше.
                    taxon = re.sub(r"^\s*"+t+r"\s*\b(\w+)\b.*", r"\1", paragraphStr, flags=re.IGNORECASE)
                    #print(taxon)
                    taxonBreadcrumbs[index] = taxon
                    # А если надо подняться наверх, Пр. на уровень семейства, меняем название семейства, всё, что ниже — стираем.
                    # Если проваливаемся сразу на несколько уровней, то они остаются пустыми.
                    taxonBreadcrumbs = taxonBreadcrumbs[:index+1]
                    taxonBreadcrumbs.extend([""]*(len(taxons)-index-1))
                    #print(taxonBreadcrumbs)
                    break;


#for e in tree.cssselect("table>tr"):
#    if(len(e)==3):
#        pass
        #e[0].text = "111"

#display(HTML(str(lxml.html.tostring(tree)))) работает, но выводить в ХТМЛ кучу \n\t
        
#out = open('out.tsv', 'w')
#result = lxml.html.tostring(tree, pretty_print=True, method="html")
#out.write(outCSV)
#out.close()

outHTML = open('Primat_s_debug.html', 'w')
result = lxml.html.tostring(tree, pretty_print=True, method="html", encoding='unicode')
#print(result[:30000])
outHTML.write(result)
outHTML.close()

[]
